In [ ]:
from nbdev import *

%nbdev_default_export retrieval

Cells will be exported to istatapi.retrieval,
unless a different module is specified after an export flag: `%nbdev_export special.module`


In [ ]:
%nbdev_hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
%nbdev_export
from istatapi.discovery import DataSet
from istatapi.base import ISTAT
import pandas as pd
import io
from fastcore.test import *

UsageError: Line magic function `%nbdev_export` not found.


# Retrieval

> Functions used to retrieve data from ISTAT datasets.

In [ ]:
%nbdev_export
RESOURCE = "data"
# TODO: accept json response as well (?)


def get_data(dataset: DataSet, **kwargs):
    "returns a dataframe of the filitered 'dataset'"
    flowRef = dataset.identifiers["df_id"]
    filters = dataset.filters
    key = make_url_key(filters)
    path_parts = [RESOURCE, flowRef, key]
    path = "/".join(path_parts)
    request = ISTAT()
    response = request._request(path, headers={"Accept": "text/csv"})
    df = pd.read_csv(io.StringIO(response.text))

    if "TIME_PERIOD" in df.columns:
        df["TIME_PERIOD"] = pd.to_datetime(
            df["TIME_PERIOD"], infer_datetime_format=True
        )
        df = df.sort_values(by=["TIME_PERIOD"])

    return df


def make_url_key(filters: dict):
    key = ""

    for i, filter_tuple in enumerate(filters.items()):

        filter = filter_tuple[0]
        filter_value = filter_tuple[1]

        # add a + and convert to str
        if type(filter_value) == list:
            filter_value = "+".join(filter_value)

        if i != 0:
            if list(filters.values())[i - 1] != ".":
                filter_value = "." + filter_value

        key += filter_value

    return key

In [ ]:
show_doc(get_data)

<h4 id="get_data" class="doc_header"><code>get_data</code><a href="__main__.py#L6" class="source_link" style="float:right">[source]</a></h4>

> <code>get_data</code>(**`dataset`**:[`DataSet`](/istatapi/discovery#DataSet), **\*\*`kwargs`**)

returns a dataframe of the filitered 'dataset'

In [ ]:
#initiate the dataset
dataset = DataSet(dataflow_identifier="139_176")

#define some filters
freq = "M"
tipo_dato=["ISAV", "ESAV"]
paese_partner="WORLD"

#set filters
dataset.set_filters(freq=freq, tipo_dato=tipo_dato, paese_partner=paese_partner)
#return dataset
unempl_df = get_data(dataset)
unempl_df.head()
test_eq(unempl_df['FREQ'].unique(), ['M'])
test_eq(unempl_df['TIPO_DATO'].unique().sort(), ["ISAV", "ESAV"].sort())

{'FREQ': 'M', 'MERCE_ATECO_2007': '.', 'PAESE_PARTNER': 'WORLD', 'ITTER107': '.', 'TIPO_DATO': ['ISAV', 'ESAV']}


In [ ]:
%nbdev_hide
export.notebook2script()

Converted 00_base.ipynb.
Converted 01_discovery.ipynb.
Converted 02_utils.ipynb.
Converted 03_retrieval.ipynb.
Converted index.ipynb.
